**Analysis of effective masses for shear force and complex force**

In [1]:
import sys
sys.path.append('../..')

import numpy as np

from wobble.pure import PureMA

ma=PureMA('test', 3, 
                     mesh_file='../sample_data_files/mesh_files/beam.msh',
                 material_file='../sample_data_files/material_files/steel.dat',
                      num_modes=1626,
                     force_path='../sample_data_files/force_files/pure_shear_500.txt')
ma.initialize_model()
overload_bc_dicts=[{'type': 'FixedValue', 'value': 0.0, 'axis': 'x', 'group': 'right'},
                    {'type': 'FixedValue', 'value': 0.0, 'axis': 'y', 'group': 'right'},
                    {'type': 'FixedValue', 'value': 0.0, 'axis': 'z', 'group': 'right'}]
ma.assemble_stiffness(overload_bc_dicts=overload_bc_dicts)
ma.assemble_mass()
ma.find_eigenmodes(eigenmode_path='../sample_data_files/eigenmode_files/clamped_modes.csv')
ma.project_force()

In [2]:
tot_mass=np.abs(((ma.eigenvectors.T@ma.m_star@ma.unprojected_force[ma.blocked_dof_mask, :][:,0])/ma.eigenvalues)).sum()
eff_mass=np.abs((ma.eigenvectors.T@ma.m_star@ma.unprojected_force[ma.blocked_dof_mask, :][:,0])/ma.eigenvalues)
for i in range(10):
    print(r"{0} & {1:.0f} & {2:.2e} & {3:.2f}\% \\".format(i+1, ma.eigenvalues[i], eff_mass[i], eff_mass[i]/tot_mass*100))
    
print(r"Total & - & {:.2e} & 100\% \\".format(tot_mass))
print(r"\hline")

1 & 39224 & 1.62e-03 & 94.91\% \\
2 & 39237 & 1.65e-05 & 0.97\% \\
3 & 1084895 & 6.68e-06 & 0.39\% \\
4 & 1198351 & 3.34e-05 & 1.96\% \\
5 & 1200573 & 7.43e-08 & 0.00\% \\
6 & 2467403 & 6.19e-12 & 0.00\% \\
7 & 7095980 & 3.10e-06 & 0.18\% \\
8 & 7102093 & 3.72e-08 & 0.00\% \\
9 & 9815132 & 7.14e-07 & 0.04\% \\
10 & 20273710 & 1.95e-07 & 0.01\% \\
Total & - & 1.71e-03 & 100\% \\
\hline


In [3]:
from wobble.rb_simple import SimpleRB

sim=SimpleRB('Double_RB', 3, 0.1, 200,
                 mesh_file='../sample_data_files/mesh_files/beam.msh',
                 material_file='../sample_data_files/material_files/steel.dat',
                num_modes=10,
                force_path='../sample_data_files/force_files/pure_rotation_500.txt')

sim.set_up()
sim.set_initial_conditions()
sim.handle_forces(eigenmode_path='../sample_data_files/eigenmode_files/rb_modes.csv')

In [4]:
tot_mass=np.abs(((sim.eigenvectors.T@sim.m_star@sim.unprojected_force[sim.blocked_dof_mask, :][:,0])/sim.eigenvalues)).sum()
eff_mass=np.abs((sim.eigenvectors.T@sim.m_star@sim.unprojected_force[sim.blocked_dof_mask, :][:,0])/sim.eigenvalues)
for i in range(10):
    print(r"{0} & {1:.0f} & {2:.2e} & {3:.2f}\% \\".format(i+1, sim.eigenvalues[i], eff_mass[i], eff_mass[i]/tot_mass*100))
    
print(r"Total & - & {:.2e} & 100\% \\".format(tot_mass))
print(r"\hline")

1 & 1337869 & 3.27e-06 & 27.87\% \\
2 & 1339773 & 8.24e-08 & 0.70\% \\
3 & 4325042 & 1.75e-06 & 14.85\% \\
4 & 7646647 & 5.78e-06 & 49.24\% \\
5 & 7652958 & 4.04e-07 & 3.44\% \\
6 & 9869694 & 4.43e-10 & 0.00\% \\
7 & 17392735 & 3.93e-07 & 3.34\% \\
8 & 21995779 & 5.66e-08 & 0.48\% \\
9 & 22077620 & 4.41e-09 & 0.04\% \\
10 & 39486567 & 4.18e-09 & 0.04\% \\
Total & - & 1.17e-05 & 100\% \\
\hline
